# G16 data

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Read in data from specified .csv file
df = pd.read_csv('New camera image data - Sheet1.csv')
data = df.values

## Comment in/out next line to see formatting of data ###
# print(df.loc[0][:])

In [11]:
### Constants ###
N = 16   # number of images in Powershot G16 Data
H_V_lattice = 3.493569812  #Reciprocal distance between Horizontal and Vertical Cu100 lattice points
D_lattice = 2.470326905    #Reciprocal distance between Diagonal Cu100 lattice points

In [12]:
### Data Values ###
electron_energy = data[:,1]
sqrt_E = np.sqrt(electron_energy)   #Used as x-values for line fit

opening_AU = data[:,2]     #Measured screen diameter in Arbitrary Units
opening_AU_m = np.mean(opening_AU)     #Average of screen diameter in AU
error_opening_AU = np.std(opening_AU)/opening_AU    #Fractional error in opening

#Separate data into separate sections
H_data = data[:,5:13]
# print(H_data[0,:])
V_data = data[:,16:24]
# print(V_data[:,3])
D_data = data[:,27:]
# print(D_data[0,:])

##need to fix first couple data points of diagonal data (divide spacing by 2 for first 4 measurements)
##due to a difference in measuring

In [13]:
### Fix first four data points
D_data[:5,1] = .5*D_data[:5,1]
D_data[:5,5] = .5*D_data[:5,5]
# print(D_data[:6,5]-D_data[:6,1])

In [14]:
### Calculated Values ###
#Screen opening and opening error
def recip_openingANDerror(lattice, data, Opening_A, Error_opening_AU):
    spacing_AU = data[:,5] - data[:,1]         #Calculate distance between points (Right_x_position - Left_x_position)
    opening = lattice*(Opening_A/spacing_AU)   #Calculate opening size in reciprocal space (lattice spacing*(Opening[AU]/Lattice spacing[AU]))
    d_left = .5*(data[:,3] - data[:,2])        #absolute error in left point .5*(width of peak at half max)
    d_right = .5*(data[:,7] - data[:,6])       #absolute error in right point .5*(width of peak at half max)
    d_spacing = np.sqrt(np.square(d_left) + np.square(d_right))   #absolute error in spacing (addition of absolute error in quadrature)
    frac_error = np.sqrt(np.square(d_spacing/spacing_AU) + np.square(Error_opening_AU/Opening_A))   #fractional error in spacing
    error_recip = frac_error*opening           #fractional error in opening calculation
    return opening, error_recip

#least squares fit with error in slope and intercept
##details of calculation in next cell
def w_lst_sq(opening_recip, error_recip, e_energy, sqrt_e):
    weights = 1/np.square(error_recip)         #weights for line fit from error in opening
    Delta = np.sum(weights)*np.sum(weights*e_energy)-np.square(np.sum(weights*sqrt_e))
    intercept = ((np.sum(weights*e_energy)*np.sum(weights*opening_recip))
                 -(np.sum(weights*sqrt_e)*np.sum(weights*sqrt_e*opening_recip)))/Delta
    slope = (np.sum(weights)*np.sum(weights*sqrt_e*opening_recip)-np.sum(weights*sqrt_e)*np.sum(weights*opening_recip))/Delta
    intercept_error = np.sqrt(np.sum(weights*e_energy)/Delta)
    slope_error = np.sqrt(np.sum(weights)/Delta)
    return slope, slope_error, intercept, intercept_error

$$ B = \frac{\sum{w  E} \sum{w  o} - \sum w\sqrt{E} \sum w o \sqrt{E} }{\Delta} $$

$$ A = \frac{\sum{w} \sum{w o \sqrt{E}}-\sum{w\sqrt e} \sum{w o}}{\Delta} $$

Where 
$$ \Delta = \sum{w} \sum{w E}- (\sum w\sqrt{e})^2 $$
And 
$$ Y = Ax + B $$

In [17]:
H_recip_opening, H_opening_error = recip_openingANDerror(H_V_lattice, H_data, opening_AU, error_opening_AU)
H1_slope, H1_slope_error, H1_intercept, H1_intercept_error = w_lst_sq(H_recip_opening, H_opening_error, electron_energy, sqrt_E)
print('H Slope + error', H1_slope, H1_slope_error, 'Intercept + error:', H1_intercept, H1_intercept_error)

V_recip_opening, V_opening_error = recip_openingANDerror(H_V_lattice, V_data, opening_AU, error_opening_AU)
V1_slope, V1_slope_error, V1_intercept, V1_intercept_error = w_lst_sq(V_recip_opening, V_opening_error, electron_energy, sqrt_E)
print('V Slope + error', V1_slope, V1_slope_error, 'Intercept + error:', V1_intercept, V1_intercept_error)

D_recip_opening, D_opening_error = recip_openingANDerror(D_lattice, D_data, opening_AU, error_opening_AU)
D1_slope, D1_slope_error, D1_intercept, D1_intercept_error = w_lst_sq(D_recip_opening, D_opening_error, electron_energy, sqrt_E)
print('D Slope + error', D1_slope, D1_slope_error, 'Intercept + error:', D1_intercept, D1_intercept_error)

H Slope + error 0.7702071387066763 0.03752820129150288 Intercept + error: 0.5130240351966252 0.3349668602467557
V Slope + error 0.7635032078814743 0.049907469612732305 Intercept + error: 0.6476329112772109 0.4537175399649789
D Slope + error 0.7776123059603269 0.05714997235386788 Intercept + error: 0.498209224988452 0.5071268144172828


# Old camera data

In [20]:
#Read in data from old camera, same format as above
df2 = pd.read_csv('Old Camera image data - Sheet1.csv')
data2 = df2.values

In [21]:
### Data Values ###
#all values same format as above
N2 = 9
electron_energy2 = data2[:,1]
sqrt_E2 = np.sqrt(electron_energy2)

opening_AU2 = data2[:,2]
opening_AU_m2 = np.mean(opening_AU2)
error_opening_AU2 = np.std(opening_AU2)/opening_AU2


H_data2 = data2[:,5:13]
# print(H_data[0,:])
V_data2 = data2[:,16:24]
# print(V_data[:,3])
D_data2 = data2[:,27:]
# print(D_data[0,:])

In [25]:
H_recip_opening2, H_opening_error2 = recip_openingANDerror(H_V_lattice, H_data2, opening_AU2, error_opening_AU2)
H2_slope, H2_slope_error, H2_intercept, H2_intercept_error = w_lst_sq(H_recip_opening2, H_opening_error2, electron_energy2, sqrt_E2)
print('H2 Slope + error', H2_slope, H2_slope_error, 'Intercept + error:', H2_intercept, H2_intercept_error)

V_recip_opening2, V_opening_error2 = recip_openingANDerror(H_V_lattice, V_data2, opening_AU2, error_opening_AU2)
V2_slope, V2_slope_error, V2_intercept, V2_intercept_error = w_lst_sq(V_recip_opening2, V_opening_error2, electron_energy2, sqrt_E2)
print('V2 Slope + error', V2_slope, V2_slope_error, 'Intercept + error:', V2_intercept, V2_intercept_error)

#Diagonal seems to be not working, need to diagnose
D_recip_opening2, D_opening_error2 = recip_openingANDerror(D_lattice, D_data2, opening_AU2, error_opening_AU2)
D2_slope, D2_slope_error, D2_intercept, D2_intercept_error = w_lst_sq(D_recip_opening2, D_opening_error2, electron_energy2, sqrt_E2)
# print('D2 Slope + error', D2_slope, D2_slope_error, 'Intercept + error:', D2_intercept, D2_intercept_error)


H2 Slope + error 0.7324657707749295 0.10906720312808338 Intercept + error: 0.49131342902567215 1.019045878458775
V2 Slope + error 0.7436281659570254 0.1157452499696649 Intercept + error: 0.44442158004113647 1.0859478181653865


## Error calculations

In [28]:
#Calculate spacing between measured points in Arbitrary Units and fractional error
def spacing_AU(data):  
    meas_spacing = data[:,5] - data[:,1]
    spacing_frac_error = np.sqrt(np.square(.5*(data[:,3] - data[:,2])) + np.square(.5*(data[:,6] - data[:,5])))/meas_spacing
    return meas_spacing, spacing_frac_error

#Fractional error in line fit parameters A and B
def frac_axb(A, d_A, B, d_B, sqrt_e):
    frac = np.sqrt(np.square(sqrt_e*d_A) + np.square(d_B))/(A*sqrt_e + B)
    return frac

#Calculate fractional error in opening calculation from line fit
def frac_error(frac_ab, frac_open, frac_space): 
    tot_frac_error = np.sqrt(np.square(frac_ab) + np.square(frac_open) + np.square(frac_space))
    return tot_frac_error

## Spacing calculations

In [33]:
#Calculate reciprocal spacing between measured points using fit line
def spacing_calc(energy, meas_opening, data):
    meas_spacing = data[:,5] - data[:,1]
    ratio = meas_spacing/meas_opening
    direction = str(input('Which direction? H/V/D (case sensitive)'))
    if direction == 'H':
        opening_func = 0.7707384404775303*np.sqrt(energy) + 0.507881158392666
    elif direction == 'V':
        opening_func = 0.7635937144289188*np.sqrt(energy) + 0.6470749251271875
    else:
        opening_func = 0.7781788677967666*np.sqrt(energy) + 0.4939931209205846
    
    spacing = ratio*opening_func
    return spacing

In [31]:
H_spacing = spacing_calc(electron_energy, opening_AU, H_data)
H2_spacing = spacing_calc(electron_energy2, opening_AU2, H_data2)

Which direction? H/V/D (case sensitive)H
Which direction? H/V/D (case sensitive)H


In [32]:
V_spacing = spacing_calc(electron_energy, opening_AU, V_data)
V2_spacing = spacing_calc(electron_energy2, opening_AU2, V_data2)

Which direction? H/V/D (case sensitive)V
Which direction? H/V/D (case sensitive)V


In [ ]:
H_frac_axb = frac_axb(H1_slope, H1_slope_error, H1_intercept, H1_intercept_error, sqrt_E)
H_spacing_meas, H_frac_spacing = spacing_AU(H_data)
H_frac_error = frac_error(H_frac_axb, error_opening_AU, H_frac_spacing)
H_error = H_frac_error*H_spacing

In [ ]:
V_frac_axb = frac_axb(V1_slope, V1_slope_error, V1_intercept, V1_intercept_error, sqrt_E)
V_spacing_meas, V_frac_spacing = spacing_AU(V_data)
V_frac_error = frac_error(V_frac_axb, error_opening_AU, V_frac_spacing)
V_error = V_frac_error*V_spacing

In [ ]:
H2_frac_axb = frac_axb(H2_slope, H2_slope_error, H2_intercept, H2_intercept_error, sqrt_E2)
H2_spacing_meas, H2_frac_spacing = spacing_AU(H_data2)
H2_frac_error = frac_error(H2_frac_axb, error_opening_AU2, H2_frac_spacing)
H2_error = H2_frac_error*H2_spacing

V2_frac_axb = frac_axb(V2_slope, V2_slope_error, V2_intercept, V2_intercept_error, sqrt_E2)
V2_spacing_meas, V2_frac_spacing = spacing_AU(V_data2)
V2_frac_error = frac_error(V2_frac_axb, error_opening_AU2, V2_frac_spacing)
V2_error = V2_frac_error*V2_spacing

### Compare Fit1 and Fit2

In [ ]:
print('slope difference', H1_slope - H2_slope, 'intercept difference', H1_intercept - H2_intercept)
print('slope difference', V1_slope - V2_slope, 'intercept difference', V1_intercept - V2_intercept)

In [ ]:
#plot lines on top of each other with shaded uncertainties

x1 = sqrt_E
x2 = sqrt_E2
y1 = H1_slope*x1 + H1_intercept
y2 = H2_slope*x2 + H2_intercept

plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(x1, y1, 'k', color='#CC4F1B', label = 'fit1')
plt.fill_between(x1, y1-H_error, y1+H_error, alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')

plt.plot(x2, y2, 'k', color='#1B2ACC', label = 'fit2')
plt.fill_between(x2, y2-H2_error, y2+H2_error, alpha=0.5, edgecolor='#1B2ACC', facecolor='#089FFF')
plt.legend(loc = 2)